In [1]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam,SGD
from keras.preprocessing.image import ImageDataGenerator

/home/anvesh/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [3]:
batch_size = 128
num_classes = 10
epochs = 50
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [5]:
def dense_block(input,num_filters,n_Denselayers,dropout_rate,bottleneck=True):
    temp = input
    for i in range(int(n_Denselayers)):
        if bottleneck:
            BatchNorm = BatchNormalization()(temp)
            relu = Activation('relu')(BatchNorm)
            Conv2D_BottleNeck = Conv2D(4*num_filters, (1,1), use_bias=False ,padding='same')(relu)
            if dropout_rate>0:
                Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
            BatchNorm = BatchNormalization()(Conv2D_BottleNeck)
        else:
            BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
            Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        temp = concat
    return temp

In [6]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(input.shape[3].value *compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [7]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [8]:
n_channels = 2 * num_filter 
n_Denselayers = 16

In [9]:

input = Input(shape=(img_height, img_width, channel,))
first_layer = Conv2D(filters=n_channels,kernel_size=(3,3),padding='same')(input)

First_Block = dense_block(first_layer, num_filter,n_Denselayers, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = dense_block(First_Transition, num_filter,n_Denselayers, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = dense_block(Second_Transition, num_filter,n_Denselayers, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter,dropout_rate)

output = output_layer(Third_Transition)

In [10]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 24)   672         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 24)   96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 24)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [11]:
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=0.1,momentum=0.9,decay=0.0001,nesterov=True),
              metrics=['accuracy'])

In [12]:
datagen = ImageDataGenerator(rotation_range=40,horizontal_flip=True)

In [13]:
datagen.fit(x_train)

In [14]:
train_gen=datagen.flow(x_train,y_train,batch_size=128)

In [33]:
model.fit_generator(train_gen,
                    epochs=10,
                    steps_per_epoch=1000,
                    verbose=1,
                    validation_data=(x_test, y_test))

Epoch 1/10
1000/1000 [==============================] - 218s 218ms/step - loss: 1.5071 - acc: 0.4518 - val_loss: 1.6255 - val_acc: 0.4481
Epoch 2/10
1000/1000 [==============================] - 179s 179ms/step - loss: 1.1781 - acc: 0.5777 - val_loss: 1.5127 - val_acc: 0.5428
Epoch 3/10
1000/1000 [==============================] - 179s 179ms/step - loss: 1.0225 - acc: 0.6358 - val_loss: 1.0690 - val_acc: 0.6452
Epoch 4/10
1000/1000 [==============================] - 178s 178ms/step - loss: 0.9298 - acc: 0.6710 - val_loss: 1.0539 - val_acc: 0.6607
Epoch 5/10
1000/1000 [==============================] - 179s 179ms/step - loss: 0.8665 - acc: 0.6953 - val_loss: 0.9308 - val_acc: 0.6983
Epoch 6/10
1000/1000 [==============================] - 179s 179ms/step - loss: 0.8152 - acc: 0.7134 - val_loss: 0.9786 - val_acc: 0.6910
Epoch 7/10
1000/1000 [==============================] - 179s 179ms/step - loss: 0.7744 - acc: 0.7278 - val_loss: 0.8657 - val_acc: 0.7211
Epoch 8/10
1000/1000 [============

In [34]:
model.save_weights("DNST_model_800_aug.h5")
print("Saved model to disk")

Saved model to disk


In [35]:
model.fit_generator(train_gen,
                    epochs=10,
                    steps_per_epoch=1000,
                    verbose=1,
                    validation_data=(x_test, y_test))

Epoch 1/10
1000/1000 [==============================] - 177s 177ms/step - loss: 0.6632 - acc: 0.7679 - val_loss: 0.7279 - val_acc: 0.7706
Epoch 2/10
1000/1000 [==============================] - 178s 178ms/step - loss: 0.6435 - acc: 0.7742 - val_loss: 0.8088 - val_acc: 0.7549
Epoch 3/10
1000/1000 [==============================] - 178s 178ms/step - loss: 0.6254 - acc: 0.7818 - val_loss: 0.7408 - val_acc: 0.7723
Epoch 4/10
1000/1000 [==============================] - 179s 179ms/step - loss: 0.6075 - acc: 0.7874 - val_loss: 0.7025 - val_acc: 0.7833
Epoch 5/10
1000/1000 [==============================] - 179s 179ms/step - loss: 0.5969 - acc: 0.7905 - val_loss: 0.6390 - val_acc: 0.7923
Epoch 6/10
1000/1000 [==============================] - 179s 179ms/step - loss: 0.5827 - acc: 0.7962 - val_loss: 0.7183 - val_acc: 0.7732
Epoch 7/10
1000/1000 [==============================] - 179s 179ms/step - loss: 0.5676 - acc: 0.8015 - val_loss: 0.7132 - val_acc: 0.7822
Epoch 8/10
1000/1000 [============

In [36]:
model.save_weights("DNST_model_800_aug_1.h5")
print("Saved model to disk")

Saved model to disk


# next train

In [15]:
model.load_weights('DNST_model_800_aug_1.h5')

In [16]:
model.fit_generator(train_gen,
                    epochs=10,
                    steps_per_epoch=1000,
                    verbose=1,
                    validation_data=(x_test, y_test))

Epoch 1/10
1000/1000 [==============================] - 202s 202ms/step - loss: 0.5707 - acc: 0.8002 - val_loss: 0.6590 - val_acc: 0.7952
Epoch 2/10
1000/1000 [==============================] - 174s 174ms/step - loss: 0.5521 - acc: 0.8070 - val_loss: 0.7005 - val_acc: 0.7872
Epoch 3/10
1000/1000 [==============================] - 174s 174ms/step - loss: 0.5261 - acc: 0.8160 - val_loss: 0.6660 - val_acc: 0.7970
Epoch 4/10
1000/1000 [==============================] - 174s 174ms/step - loss: 0.5085 - acc: 0.8217 - val_loss: 0.6463 - val_acc: 0.8053
Epoch 5/10
1000/1000 [==============================] - 175s 175ms/step - loss: 0.4917 - acc: 0.8268 - val_loss: 0.6934 - val_acc: 0.7915
Epoch 6/10
1000/1000 [==============================] - 174s 174ms/step - loss: 0.4764 - acc: 0.8338 - val_loss: 0.6407 - val_acc: 0.8093
Epoch 7/10
1000/1000 [==============================] - 175s 175ms/step - loss: 0.4648 - acc: 0.8371 - val_loss: 0.5004 - val_acc: 0.8446
Epoch 8/10
1000/1000 [============

In [17]:
model.save_weights("DNST_model_800_aug_2.h5")
print("Saved model to disk")

Saved model to disk


# next

In [18]:
model.fit_generator(train_gen,
                    epochs=10,
                    steps_per_epoch=1000,
                    verbose=1,
                    validation_data=(x_test, y_test))

Epoch 1/10
1000/1000 [==============================] - 174s 174ms/step - loss: 0.4269 - acc: 0.8508 - val_loss: 0.5407 - val_acc: 0.8428
Epoch 2/10
1000/1000 [==============================] - 174s 174ms/step - loss: 0.4185 - acc: 0.8525 - val_loss: 0.5936 - val_acc: 0.8254
Epoch 3/10
1000/1000 [==============================] - 174s 174ms/step - loss: 0.4096 - acc: 0.8562 - val_loss: 0.5581 - val_acc: 0.8290
Epoch 4/10
1000/1000 [==============================] - 176s 176ms/step - loss: 0.3948 - acc: 0.8604 - val_loss: 0.6568 - val_acc: 0.8128
Epoch 6/10
1000/1000 [==============================] - 175s 175ms/step - loss: 0.3932 - acc: 0.8626 - val_loss: 0.5625 - val_acc: 0.8337
Epoch 7/10
1000/1000 [==============================] - 174s 174ms/step - loss: 0.3889 - acc: 0.8631 - val_loss: 0.5730 - val_acc: 0.8361
Epoch 8/10
1000/1000 [==============================] - 174s 174ms/step - loss: 0.3809 - acc: 0.8664 - val_loss: 0.5249 - val_acc: 0.8430
Epoch 9/10
1000/1000 [============

In [19]:
model.save_weights("DNST_model_800_aug_3.h5")
print("Saved model to disk")

Saved model to disk


# next

In [20]:
model.fit_generator(train_gen,
                    epochs=10,
                    steps_per_epoch=1000,
                    verbose=1,
                    validation_data=(x_test, y_test))

Epoch 1/10
1000/1000 [==============================] - 174s 174ms/step - loss: 0.3706 - acc: 0.8695 - val_loss: 0.4922 - val_acc: 0.8540
Epoch 2/10
1000/1000 [==============================] - 174s 174ms/step - loss: 0.3643 - acc: 0.8713 - val_loss: 0.5073 - val_acc: 0.8482
Epoch 3/10
1000/1000 [==============================] - 174s 174ms/step - loss: 0.3598 - acc: 0.8737 - val_loss: 0.4894 - val_acc: 0.8520
Epoch 4/10
1000/1000 [==============================] - 174s 174ms/step - loss: 0.3529 - acc: 0.8756 - val_loss: 0.4920 - val_acc: 0.8543
Epoch 5/10
1000/1000 [==============================] - 177s 177ms/step - loss: 0.3504 - acc: 0.8775 - val_loss: 0.4922 - val_acc: 0.8539
Epoch 6/10
1000/1000 [==============================] - 175s 175ms/step - loss: 0.3464 - acc: 0.8783 - val_loss: 0.5286 - val_acc: 0.8472
Epoch 7/10
1000/1000 [==============================] - 174s 174ms/step - loss: 0.3462 - acc: 0.8778 - val_loss: 0.5335 - val_acc: 0.8452
Epoch 8/10
1000/1000 [============

In [22]:
model.save_weights("DNST_model_800_aug_4.h5")
print("Saved model to disk")

Saved model to disk


# next

In [23]:
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=0.001,momentum=0.9,decay=0.0001,nesterov=True),
              metrics=['accuracy'])

In [24]:
model.fit_generator(train_gen,
                    epochs=10,
                    steps_per_epoch=1000,
                    verbose=1,
                    validation_data=(x_test, y_test))

Epoch 1/10
1000/1000 [==============================] - 202s 202ms/step - loss: 0.3228 - acc: 0.8857 - val_loss: 0.4778 - val_acc: 0.8582
Epoch 2/10
1000/1000 [==============================] - 175s 175ms/step - loss: 0.3186 - acc: 0.8875 - val_loss: 0.4776 - val_acc: 0.8614
Epoch 3/10
1000/1000 [==============================] - 175s 175ms/step - loss: 0.3162 - acc: 0.8887 - val_loss: 0.4756 - val_acc: 0.8628
Epoch 4/10
1000/1000 [==============================] - 175s 175ms/step - loss: 0.3178 - acc: 0.8879 - val_loss: 0.4844 - val_acc: 0.8553
Epoch 5/10
1000/1000 [==============================] - 175s 175ms/step - loss: 0.3118 - acc: 0.8902 - val_loss: 0.4753 - val_acc: 0.8600
Epoch 6/10
1000/1000 [==============================] - 175s 175ms/step - loss: 0.3125 - acc: 0.8895 - val_loss: 0.4817 - val_acc: 0.8576
Epoch 7/10
1000/1000 [==============================] - 175s 175ms/step - loss: 0.3091 - acc: 0.8903 - val_loss: 0.4817 - val_acc: 0.8572
Epoch 8/10
1000/1000 [============

In [25]:
model.save_weights("DNST_model_800_aug_5.h5")
print("Saved model to disk")

Saved model to disk


In [26]:
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=0.0001,momentum=0.9,decay=0.0001,nesterov=True),
              metrics=['accuracy'])

In [ ]:
model.fit_generator(train_gen,
                    epochs=10,
                    steps_per_epoch=1000,
                    verbose=1,
                    validation_data=(x_test, y_test))

Epoch 1/10
1000/1000 [==============================] - 205s 205ms/step - loss: 0.3053 - acc: 0.8920 - val_loss: 0.4721 - val_acc: 0.8612
Epoch 2/10
1000/1000 [==============================] - 175s 175ms/step - loss: 0.3066 - acc: 0.8931 - val_loss: 0.4719 - val_acc: 0.8630
Epoch 3/10
1000/1000 [==============================] - 176s 176ms/step - loss: 0.3032 - acc: 0.8934 - val_loss: 0.4719 - val_acc: 0.8628
Epoch 4/10
1000/1000 [==============================] - 175s 175ms/step - loss: 0.3042 - acc: 0.8929 - val_loss: 0.4726 - val_acc: 0.8615
Epoch 5/10
1000/1000 [==============================] - 176s 176ms/step - loss: 0.3038 - acc: 0.8932 - val_loss: 0.4721 - val_acc: 0.8622
Epoch 6/10
1000/1000 [==============================] - 175s 175ms/step - loss: 0.3056 - acc: 0.8925 - val_loss: 0.4728 - val_acc: 0.8613
Epoch 7/10
1000/1000 [==============================] - 176s 176ms/step - loss: 0.3023 - acc: 0.8927 - val_loss: 0.4735 - val_acc: 0.8620
Epoch 8/10
1000/1000 [============

In [ ]:
model.save_weights("DNST_model_800_aug_6.h5")
print("Saved model to disk")

# next

In [48]:
model.load_weights("DNST_model_800_aug_6.h5")

In [49]:
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=0.1),
              metrics=['accuracy'])

In [50]:
model.fit_generator(train_gen,
                    epochs=20,
                    steps_per_epoch=1000,
                    verbose=1,
                    validation_data=(x_test, y_test))

Epoch 1/20
1000/1000 [==============================] - 252s 252ms/step - loss: 0.3978 - acc: 0.8598 - val_loss: 0.5758 - val_acc: 0.8271
Epoch 2/20
1000/1000 [==============================] - 181s 181ms/step - loss: 0.3866 - acc: 0.8637 - val_loss: 0.7250 - val_acc: 0.7997
Epoch 3/20
1000/1000 [==============================] - 179s 179ms/step - loss: 0.3785 - acc: 0.8673 - val_loss: 0.5618 - val_acc: 0.8427
Epoch 4/20
1000/1000 [==============================] - 178s 178ms/step - loss: 0.3671 - acc: 0.8713 - val_loss: 0.6256 - val_acc: 0.8255
Epoch 5/20
1000/1000 [==============================] - 178s 178ms/step - loss: 0.3599 - acc: 0.8729 - val_loss: 0.5690 - val_acc: 0.8441
Epoch 6/20
1000/1000 [==============================] - 179s 179ms/step - loss: 0.3492 - acc: 0.8767 - val_loss: 0.5856 - val_acc: 0.8392
Epoch 7/20
1000/1000 [==============================] - 182s 182ms/step - loss: 0.3455 - acc: 0.8779 - val_loss: 0.5490 - val_acc: 0.8502
Epoch 8/20
1000/1000 [============

In [51]:
model.save_weights("DNST_model_800_aug_7.h5")
print("Saved model to disk")

Saved model to disk


In [18]:
model.fit_generator(train_gen,
                    epochs=10,
                    steps_per_epoch=1000,
                    verbose=1,
                    validation_data=(x_test, y_test))

Epoch 1/10
1000/1000 [==============================] - 191s 191ms/step - loss: 0.2438 - acc: 0.9142 - val_loss: 0.5254 - val_acc: 0.8662
Epoch 2/10
1000/1000 [==============================] - 191s 191ms/step - loss: 0.2343 - acc: 0.9178 - val_loss: 0.4646 - val_acc: 0.8779
Epoch 3/10
1000/1000 [==============================] - 191s 191ms/step - loss: 0.2264 - acc: 0.9206 - val_loss: 0.5000 - val_acc: 0.8716
Epoch 4/10
1000/1000 [==============================] - 191s 191ms/step - loss: 0.2171 - acc: 0.9230 - val_loss: 0.4628 - val_acc: 0.8860
Epoch 5/10
1000/1000 [==============================] - 191s 191ms/step - loss: 0.2109 - acc: 0.9250 - val_loss: 0.5284 - val_acc: 0.8730
Epoch 6/10
1000/1000 [==============================] - 192s 192ms/step - loss: 0.2037 - acc: 0.9277 - val_loss: 0.4825 - val_acc: 0.8817
Epoch 7/10
1000/1000 [==============================] - 191s 191ms/step - loss: 0.1962 - acc: 0.9308 - val_loss: 0.5031 - val_acc: 0.8759
Epoch 8/10
1000/1000 [============

In [19]:
model.save_weights("DNST_model_800_aug_8.h5")

In [15]:
model.load_weights("DNST_model_800_aug_8.h5")

In [16]:
model.fit_generator(train_gen,
                    epochs=10,
                    steps_per_epoch=1000,
                    verbose=1,
                    validation_data=(x_test, y_test))

Epoch 1/10
1000/1000 [==============================] - 214s 214ms/step - loss: 0.2727 - acc: 0.9040 - val_loss: 0.6020 - val_acc: 0.8458
Epoch 2/10
1000/1000 [==============================] - 174s 174ms/step - loss: 0.2584 - acc: 0.9088 - val_loss: 0.5060 - val_acc: 0.8711
Epoch 3/10
1000/1000 [==============================] - 174s 174ms/step - loss: 0.2411 - acc: 0.9134 - val_loss: 0.6976 - val_acc: 0.8323
Epoch 4/10
1000/1000 [==============================] - 174s 174ms/step - loss: 0.2249 - acc: 0.9198 - val_loss: 0.4428 - val_acc: 0.8848
Epoch 5/10
1000/1000 [==============================] - 174s 174ms/step - loss: 0.2167 - acc: 0.9228 - val_loss: 0.5166 - val_acc: 0.8763
Epoch 6/10
1000/1000 [==============================] - 174s 174ms/step - loss: 0.2034 - acc: 0.9276 - val_loss: 0.4868 - val_acc: 0.8778
Epoch 7/10
1000/1000 [==============================] - 174s 174ms/step - loss: 0.1925 - acc: 0.9320 - val_loss: 0.4494 - val_acc: 0.8920
Epoch 8/10
1000/1000 [============

In [17]:
model.save_weights("DNST_model_800_aug_9.h5")

In [18]:
model.fit_generator(train_gen,
                    epochs=20,
                    steps_per_epoch=1000,
                    verbose=1,
                    validation_data=(x_test, y_test))

Epoch 1/20
1000/1000 [==============================] - 174s 174ms/step - loss: 0.1639 - acc: 0.9415 - val_loss: 0.5011 - val_acc: 0.8850
Epoch 2/20
1000/1000 [==============================] - 174s 174ms/step - loss: 0.1603 - acc: 0.9427 - val_loss: 0.4555 - val_acc: 0.8923
Epoch 3/20
1000/1000 [==============================] - 174s 174ms/step - loss: 0.1544 - acc: 0.9448 - val_loss: 0.4809 - val_acc: 0.8890
Epoch 4/20
1000/1000 [==============================] - 174s 174ms/step - loss: 0.1468 - acc: 0.9473 - val_loss: 0.5185 - val_acc: 0.8843
Epoch 5/20
1000/1000 [==============================] - 174s 174ms/step - loss: 0.1431 - acc: 0.9491 - val_loss: 0.4623 - val_acc: 0.8949
Epoch 6/20
1000/1000 [==============================] - 174s 174ms/step - loss: 0.1383 - acc: 0.9504 - val_loss: 0.4615 - val_acc: 0.8948
Epoch 7/20
1000/1000 [==============================] - 174s 174ms/step - loss: 0.1359 - acc: 0.9511 - val_loss: 0.4594 - val_acc: 0.8978
Epoch 8/20
1000/1000 [============

In [19]:
model.save_weights("DNST_model_800_aug_10.h5")

In [20]:
model.fit_generator(train_gen,
                    epochs=20,
                    steps_per_epoch=1000,
                    verbose=1,
                    validation_data=(x_test, y_test))

Epoch 1/20
1000/1000 [==============================] - 174s 174ms/step - loss: 0.0989 - acc: 0.9643 - val_loss: 0.4799 - val_acc: 0.9048
Epoch 2/20
1000/1000 [==============================] - 174s 174ms/step - loss: 0.0969 - acc: 0.9655 - val_loss: 0.5102 - val_acc: 0.8963
Epoch 3/20
1000/1000 [==============================] - 174s 174ms/step - loss: 0.0951 - acc: 0.9659 - val_loss: 0.4800 - val_acc: 0.9036
Epoch 4/20
1000/1000 [==============================] - 174s 174ms/step - loss: 0.0963 - acc: 0.9650 - val_loss: 0.4743 - val_acc: 0.9021
Epoch 5/20
1000/1000 [==============================] - 174s 174ms/step - loss: 0.0931 - acc: 0.9666 - val_loss: 0.5016 - val_acc: 0.8966
Epoch 6/20
1000/1000 [==============================] - 174s 174ms/step - loss: 0.0915 - acc: 0.9670 - val_loss: 0.5453 - val_acc: 0.8950
Epoch 7/20
1000/1000 [==============================] - 174s 174ms/step - loss: 0.0909 - acc: 0.9670 - val_loss: 0.4748 - val_acc: 0.9017
Epoch 8/20
1000/1000 [============

In [21]:
model.save_weights("DNST_model_800_aug_11.h5")

In [22]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 74s 1ms/step - loss: 0.0379 - acc: 0.9870 - val_loss: 0.4247 - val_acc: 0.9170
Epoch 2/50
50000/50000 [==============================] - 74s 1ms/step - loss: 0.0313 - acc: 0.9889 - val_loss: 0.4550 - val_acc: 0.9138
Epoch 3/50
50000/50000 [==============================] - 74s 1ms/step - loss: 0.0286 - acc: 0.9903 - val_loss: 0.4362 - val_acc: 0.9178
Epoch 4/50
50000/50000 [==============================] - 74s 1ms/step - loss: 0.0278 - acc: 0.9903 - val_loss: 0.4338 - val_acc: 0.9173
Epoch 5/50
50000/50000 [==============================] - 74s 1ms/step - loss: 0.0262 - acc: 0.9910 - val_loss: 0.4394 - val_acc: 0.9184
Epoch 6/50
50000/50000 [==============================] - 74s 1ms/step - loss: 0.0245 - acc: 0.9918 - val_loss: 0.4276 - val_acc: 0.9210
Epoch 7/50
50000/50000 [==============================] - 74s 1ms/step - loss: 0.0226 - acc: 0.9924 - val_loss: 0.4340 - val_acc

In [23]:
model.save_weights("DNST_model_800_aug_12.h5")